In [139]:

# from sbhandler import *
from math import pow    
# from fastai.vision.all import *

# Needed imports
import numpy as np
import h5py

# Source code for functions from seqblock_handler.py
def load_matlab_file(mat_file, variable_name): # Note the double return statement
    """_summary_
    //UNIQUE: Only one variable for this .mat file
    
    Args:
        mat_file (string): Name of .mat file to intake (path must be given if not in current dir)
        variable_name (string): which variable from the mat file. Only one here because of the .mat file structure.

    Returns:
        h5py_object, data: Overall h5py_object and the specific variable as an nparray
    """
    h5py_object = h5py.File(mat_file, 'r')
    data = h5py_object.get(variable_name)
    data = np.array(data)
    return h5py_object, data

class DNA_SeqBlocks():
    """Mat dataset 
    Loads the mat file with DNA sequences for easy iteration through all stored DNA sequence lists of varying coverage
    """
    def __init__(self, h5py_object, data):
        """Initialize DNA_SeqBlocks() object
        Args:
            h5py_object (h5py_object): first return variable of load_matlab_file
            data (np.array): second return variable of load_matlab_file
        """
        self.h5py_object = h5py_object
        self.data = np.transpose(data)
        self.size = len(data) 
        
    def get_seqblock(self, number): # checked
        x = self.h5py_object[self.data[0, number-1]][:, :] # //UNIQUE: 0th column because .mat file, align3 variable only has one row
        return np.transpose(x)
    
    def get_coverage_count(self, number): # No need to change...
        x = self.h5py_object[self.data[0, number-1]][0,:]
        coverage_count = int((len(x)-4)/2)
        return coverage_count
    
    def get_quality(self, number, coverage_count, nuc_pos): # checked
        x = self.h5py_object[self.data[0, number-1]][nuc_pos, (1+coverage_count):(1+2*coverage_count)]
        return np.transpose(x)
    
    def get_nuc(self, number, coverage_count, nuc_pos): # checked
        x = self.h5py_object[self.data[0,number-1]][nuc_pos, 1:(coverage_count+1)]
        return np.transpose(x)
    
    def get_interp_changes(self, number, coverage_count): # checked
        x = self.h5py_object[self.data[0,number-1]][:,(1+2*coverage_count)]
        return np.transpose(x)
    
    def get_interp_mutations(self, number, coverage_count):
        x = self.h5py_object[self.data[0,number-1]][:,(3+2*coverage_count)]
        return np.transpose(x)
    
    def get_interp_consensus(self, number, coverage_count): 
        x = self.h5py_object[self.data[0,number-1]][:,(2+2*coverage_count)]
        return np.transpose(x)
    
def seqblock_parser(seqblock):
    seqblock_parsed = CleanSeqBlock()
    total_columns = len(np.transpose(seqblock)) # is there a more efficient way?
    total_rows = len(seqblock)
    number_of_reads = int((total_rows - 4)/2) # last three rows have intreptations, first row is non-mutated target sequence, and N quality scores for N reads
    
    seqblock_parsed.cov = number_of_reads;
    seqblock_parsed.len = total_columns;
    
    # print(total_columns)
    # print(total_rows)
    tag = ""
    for i in range(total_rows):
        # print(f" The sequenceblock input {seqblock}") #debug
        # print(f" The m x n size of the seqblock {seqblock.shape}") #debug
        # print(f" The ith row of the seqblock {current_seq}") #debug
        # print(f" The m x n size of the sequence {current_seq.shape}") #debug
        
        # new_format = np.chararray(total_columns) //CHANGED to string
        stop_found = False
        # new_format[:] = 'q' #debug
        
        # print(f" The 0th entry of the initialized char array {new_format[0, 0]}") #debug
        # print(f" Its row size {len(new_format[0])}")
        # if (i == 0):
        #     original = ""
        #     for j in range(total_columns):
        #         original += chr(seqblock[i,j])
                
                
        
        # if (i != 0):
        new_format = ""
        j=0
        while j < total_columns:
            # print(j)
                                    
            if stop_found is False:
                if (seqblock[i,j] == ord('T') and (j+6) < total_columns):
                    if (seqblock[i,j+1] == ord('A') and seqblock[i, j+2] == ord('A') and seqblock[i, j+3] == ord('T') and seqblock[i, j+4] == ord('A') and seqblock[i, j+5] == ord('G')):
                        new_format += 'T'
                        new_format += 'A'
                        new_format += 'A'
                        new_format += 'T'
                        new_format += 'G'
                        j = j+6
                        stop_found = True
                if stop_found is False:
                    new_format += chr(seqblock[i,j])
                    j+=1
        
            elif stop_found is True and i == 1:
                tag += chr(seqblock[i,j])
                j+=1
            else:
                j = total_columns
        if (i == 1): seqblock_parsed.barcode = tag
                
            
            
        if (i == 0):
            seqblock_parsed.target = new_format
        elif (i == total_rows - 1): # last row is mutations 'x', total_rows is one more than total index
            seqblock_parsed.interp_mutations = new_format
        elif (i == total_rows - 2): # 2nd last is subjective consensus
            seqblock_parsed.interp_consensus = new_format
        elif (i == total_rows - 3): 
            seqblock_parsed.interp_changes = new_format
        elif (i > 0 and i < number_of_reads + 1):
            seqblock_parsed.reads.append(new_format)
        else:
            seqblock_parsed.qscores.append(new_format)
    return seqblock_parsed     
    
class CleanSeqBlock():
    def __init__(self):
        self.cov, self.len, self.target, self.interp_changes, self.interp_mutations, self.interp_consensus, self.barcode = 0,0,0,0,0,0,0
        self.reads, self.qscores = [], []


In [150]:
barcodes = [len(seqblock_parser(DNA_seqs.get_seqblock(i)).barcode) for i in range(0, 1000, 50)]
(seqblock_parser(DNA_seqs.get_seqblock(650)).barcode)

'AAAAATAGAACACACGGATCCACCGGATC-AGATAACTGATCATAATCAGCCATACCACATTTGTAGAGGTTTT'

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


# Obtaining Data and Scrubbing (OS)

In [140]:

# load_matlab_file function Testing
file_name = 'consensus1.mat'
variable_of_interest = 'align3'
h5py_object, data = load_matlab_file(file_name, variable_name=variable_of_interest)

# DNA_seqs can be used to grab each raw_seqblock
DNA_seqs = DNA_SeqBlocks(h5py_object=h5py_object, data=data) 
FILE_SIZE = DNA_seqs.size #Constant

In [13]:
# Get_seqblock function Testing
test_raw_seqblock = DNA_seqs.get_seqblock(1) # Let's look at the first DNA sequence block
test_raw_seqblock[9,:] # [row, column] 

t_seqblock= seqblock_parser(test_raw_seqblock) # Let's test the parsing function to get a CleanSeqBlock

# Assess proper formatting of each aspect of the raw_seqblock
# print(f'Seq Length: {t_seqblock.len}, # of Reads: {t_seqblock.reads_count}') 
# print()
# print(f'Target sequence {t_seqblock.target} \n \n All {t_seqblock.reads_count} reads {t_seqblock.reads} \n\n Quality scores string {t_seqblock.qscores}')
# print()
# print(f'Yiyang\'s declared indels/base-call errors: {t_seqblock.interp_changes} \n Inferred consensus: {t_seqblock.interp_consensus} \n \'x\' marks the spot: {t_seqblock.interp_mutations}')

In [113]:
ordinal_stop = [ord(i) for i in 'TAATAG']
y = np.array(ordinal_stop)
y, DNA_seqs.get_seqblock(1)

(array([84, 65, 65, 84, 65, 71]),
 array([[67, 84, 65, ..., 84, 71, 65],
        [67, 84, 65, ..., 84, 71, 65],
        [67, 84, 65, ..., 84, 71, 65],
        ...,
        [32, 32, 32, ..., 32, 32, 32],
        [67, 84, 65, ..., 84, 71, 65],
        [32, 32, 32, ..., 32, 32, 32]], dtype=uint16))

# Data Exploration (E)
### How are these *Phred* quality scores structured and used?
*MAX* = 126, *MIN* = 33, **RANGE** = 93

Prob. of a base-call error (P) = `Math.pow(10, -(Q-33)/10)`

Prob. of accuracy = 1 - P

In [ ]:
#!hide
"""
max_q = []
min_q = []
for x in range(1, 350000, 500):
    seqblock = seqblock_parser(DNA_seqs.get_seqblock(x))
# for one seqblock
    for i in range(seqblock.reads_count):
        temp_list = [ord(q) for q in seqblock.qscores[i]]
        max_q.append(max(temp_list))
        min_q.append(min(temp_list))
(max(max_q)-min(min_q)), max(max_q), min(min_q)
"""
max_q = []
min_q = []
for x in range(1, 350000, 500):
    seqblock = seqblock_parser(DNA_seqs.get_seqblock(x))
# for one seqblock
    for i in range(seqblock.reads_count):
        temp_list = [ord(q) for q in seqblock.qscores[i]]
        max_q.append(max(temp_list))
        min_q.append(min(temp_list))
(max(max_q)-min(min_q)), max(max_q), min(min_q)

In [101]:

# Supposed conversion from Q score to a probability.
def prob(Qscore): 
    return pow(10, -(Qscore-33)/10)
(f'{prob(33):.2f}'), (f'{prob(126):.2e}')

('1.00', '5.01e-10')

### What is the typical coverage?   
| 1 | 2- | 3- |
| --- | --- | --- |
| 32% | 55% (+23%) | 70% (+15%) | 

**32%** have 1 coverage

**55%** have 2 or less coverage

**70%** have 3 or less coverage

I.e. **45%** have moderate to high coverage, **55%** have **LOW** coverage


**Significance of Results**: 

Majority/weighted voting can only be applied to ~45% of the dataset (3.+ coverage)

Low-coverage data can be rectified with **maximum likelihoods?**, **logistic model w/ knots** used in Li 2004?, **quality scores** (use as *baseline*)

In [ ]:
# Cell takes approx. 90 seconds to run for 355104 seqblocks
coverage_list = [DNA_seqs.get_coverage_count(x) for x in range(1,355105)]

# Extract the exact values of each coverage (max is 20)
coverage_dist = []
for i in range(1,21):
    coverage_dist.append(coverage_list.count(i))

In [ ]:
print(f'Proportion of 1-coverage seqblocks: {coverage_dist[0]/sum(coverage_dist):>12.2%}')
print(f'Proportion of 2-coverage seqblocks: {coverage_dist[1]/sum(coverage_dist):>12.2%}')
print(f'Proportion of [1,2] coverage seqblocks: {(coverage_dist[0]+coverage_dist[1])/sum(coverage_dist):>8.2%}')
print(f'Proportion of [1,2,3] coverage seqblocks: {(coverage_dist[0]+coverage_dist[1]+coverage_dist[2])/sum(coverage_dist):>6.2%}')
print(f'High coverage (6+) Proportion: {sum(coverage_dist[5:len(coverage_dist):1])/sum(coverage_dist):>17.2%}')

# Histogram of the coverage distribution
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (6,4))
ax.set_ylim(0, 120000)
plt.hist(coverage_list, bins=21, align='mid', alpha=0.9, histtype='stepfilled', color='steelblue', edgecolor='none')
ax.set_xticks(range(21))
ax.set_xlabel('# of Coverage')
ax.set_ylabel('Frequency')
ax.set_title('Coverage Distribution of consensus1.mat')

# IDXS are in Python format since enumerate starts at 0
single_coverage_idxs = [i for i, e in enumerate(coverage_list) if e == 1]
double_coverage_idxs = [i for i, e in enumerate(coverage_list) if e == 2]
triple_coverage_idxs = [i for i, e in enumerate(coverage_list) if e == 3]

high_coverage_idxs = [i for i, e in enumerate(coverage_list) if e >= 6]


### Quality of Low Coverage Reads

In [ ]:
# # Let's first grab all the attributes of each single coverage seq block... 30 minutes
# lquality = []
# lread = []
# lmutation = []
# lnotes = []
# lconsensus = []

# for i in single_coverage_idxs:
#     sb = seqblock_parser(DNA_seqs.get_seqblock(i+1))
#     lquality.append(sb.qscores)
#     lread.append(sb.reads)
#     lmutation.append(sb.interp_mutations)
#     lnotes.append(sb.interp_changes)
#     lconsensus.append(sb.interp_consensus)

In [ ]:
# # Assess the quality scores at mutation areas (first try 'x'), then try 'z', then manually find overlaps within a small range of seqblock
# idxs_of_interest = []
# seqblocks = []
# for i in range(len(single_coverage_idxs)): # using the single_coverage_idxs length gives the position of each lmutation 
#     currmut = lmutation[i]
#     idxs = [l for l, ltr in enumerate(currmut) if ltr == 'x']
#     seqblocks.append([i]*len(idxs))
#     idxs_of_interest.append(idxs)
# mutations = list(zip(seqblocks, idxs_of_interest))
# mutations

In [ ]:
# # Working with the new form
# for x,y in mutations:
#     print(y)
#     x = x[0] # how to index within lmutation
#     currmut = lmutation[x]
#     currqual = str(lquality[x])
#     print(currqual)
#     print(len(currqual))
#     qscores = [currqual[b] for b in y]
# print(qscores)

### Read Lengths and Nucleotide Frequency
- Huge spike at around ~1609 with some outliers as low as 100 (doesn't really matter.)
- Question is whether the base calls are biased (check support)

- IMPORTANT: Are there clusters in our dataset? Do certain indices have lower coverage, more inaccuracy, or different read lengths?

In [29]:
# Assessing the support of each base 
Ccount = 0
Tcount = 0
Acount = 0
Gcount = 0
indelcount = 0

for i in range(1, 355105, 50000):
    # for one seqblock
    t_seqblock = seqblock_parser(DNA_seqs.get_seqblock(i))
    for j in range(t_seqblock.reads_count):
        for i in t_seqblock.reads[j]: #Input change here
            if i == 'C':
                Ccount+=1;
            if i == 'A':
                Acount+=1;
            if i == 'T':
                Tcount +=1;
            if i == 'G':
                Gcount +=1;
            if i == '-':
                indelcount +=1;
    # print(f" C: {Ccount} \n G: {Gcount} \n A: {Acount} \n T: {Tcount} \n indels: {indelcount}")
    # print(test_parse.seq_len)
    # assert Ccount+Tcount+Gcount+Acount+indelcount == t_seqblock.len, f"Info lost in {j}th index read "
    # print(f"Success in {j}th read!")    

In [31]:
# Nucleotide Counts
Ccount, Tcount, Acount, Gcount, indelcount
total = Ccount + Tcount + Acount + Gcount + indelcount
print(f"""
      % A:  {Acount/total:.2f}
      % C:  {Ccount/total:.2f}
      % T:  {Tcount/total:.2f}
      % G:  {Gcount/total:.2f}
      % -:  {indelcount/total:.2f}
      """)

'''
% A:  0.25
% C:  0.31
% T:  0.16
% G:  0.28
for every 500th
'''


      % A:  0.25
      % C:  0.29
      % T:  0.16
      % G:  0.26
      % -:  0.04
      


'\n% A:  0.25\n% C:  0.31\n% T:  0.16\n% G:  0.28\nfor every 500th\n'

In [25]:
# Probabiliy dist of reads
# %matplotlib inline
# import matplotlib.pyplot as plt

# seq_len_dist = []
# for i in seqblocks_list:
#     seq_len_dist.append(i.seq_len)

# plt.xlabel("Sequence length")

# plt.ylabel("Frequency")
# plt.title("Prob. Dist. of first 100k sequence blocks")
# plt.hist(seq_len_dist, bins=100, range=(150, 1800), color = 'blue',
#         histtype = 'bar')

In [ ]:
# Test runtime of creating a CleanSeqBlock of all 355 104 sequence blocks

# --------------------------------------------------------------------------------------
# Load the .mat file into the DNA_SeqBlocks class (analogous to torch.utils.data.Dataset)
# ---
file_name = 'consensus1.mat'
variable_of_interest = 'align3'
h5py_object, data = load_matlab_file(file_name, variable_name=variable_of_interest)

dataset = DNA_SeqBlocks(h5py_object=h5py_object, data=data)

seqblocks_list = []
for i in range(dataset.size+1): #for all 355104 seqblocks, remember that range doesn't include last number and matlab indexing used for get_seqblock
    print(i)
    seqblock_i = dataset.get_seqblock(i) #matlab indexing to allow easy comparison
    cleanseqblock_i = seqblock_parser(seqblock_i) # creates ith CleanSeqBlock
    seqblocks_list.append(cleanseqblock_i) # to house all 355104 CleanSeqBlock(s)
print(len(seqblocks_list))
    
print(f"Total runtime to create 355104 CleanSeqBlock objects: {end-start:.1f} seconds = {(end-start)/60:.0f} minutes.")



In [ ]:
# # Using pickle to save the CleanSeqBlocks list
# import pickle

# # Let's try with the current list of 10,000 CleanSeqBlock
# with open('last_155_104.pickle', 'wb') as file_out:
#     pickle.dump(home_list_3, file_out)
    
# # unpickled_list = pickle.load(open('data.pickle','rb'))
# # print(len(unpickled_list))


## Understanding the meaning of 'z', 'x', and 'y' in dataset
> Key: Absence of quality score use
- **'z'** = failed majority voting btwn reads
- **'x'** = correct/incorrect mutation (majority voted)
- **'yyy'** = indel at non-terminating y's in at least ONE read 


### Testing the meaning of 'z'
- **Confirmed.** 'z' means a disagreement in reads (does not assess respective quality scores)

//TODO:  Resolve most of these conflicts using some architecture. E.g. error pattern from BFP reads to resolve conflicts

### Poor quality 'z' conflicts in double coverage
- About 81 674 seqblocks (~23% of 355104 seqblocks) -> ~10^3 nucleotides/seqblock * 10^4 seqblocks
- 14 978 total conflicts

- 1 765 cases of max Q score != 126 (10^3) => *1/10^4 frequency*
- 166   cases if 73+ is considered conclusive (10^2) => 99.99% accuracy, 1/10^5
- 77    cases if 63+ is considered conclusive (10^1) => 99.9% accuracy, 1/10^6

### Poor quality 'z' conflicts in triple coverage
- About 53 265 seqblocks (~15% of dataset) -> ~10^3 nuc/seqblock * 10^4 seqblocks
- 21 417  total conflicts


- 372   cases of max Q score != 126 (10^2) => *1/10^5 frequency*
- 9     cases of no max Q score 73+ (10^3) => 99.99% accuracy >> 17590, 78157, 119067, 152067, 187907, 189538, 200562, 269346, 315823
- 5     cases of no max Q score 63+ (10^0) => 99.9% accuracy >> 17590, 78157, 152067, 187907, 315823

### Poor quality 'z' conflicts in quadruple coverage
- 18 958 total conflicts

- 90 cases of no max Q score of 126
- 1 case of no max Q score of 73+ >> SB 139087, Nuc 2. Start of reads. Best to guess nonexistent mutation.
- 1 case of no max Q score of 63+

### Further?
- No reason. Clearly shows most conflicts are easily resolved through high accuracy nucleotide reads, especially when coverage >= 3. Concern is how to resolve 2-cov read conflicts at low quality.

In [80]:
tinyDS = range(1, 10_001)
totalDS = range(1, FILE_SIZE+1)


In [98]:
COV_TEST=2
CHAR_TEST='z' # Can only be y or z. Though it's currently designed for z only

test = [] 
seq = []

for i in totalDS: #whole 
    cov = DNA_seqs.get_coverage_count(i)
    if (cov == COV_TEST):  # Only care about a specific coverage
        interp = DNA_seqs.get_interp_changes(i, COV_TEST) # Now we're looking at the interp changes... eventually searching for 'z'
        for npos in range(len(interp)):
            if interp[npos] == ord(CHAR_TEST): #Oh dang there's the char at this npos... let's add an array showing the nuc reads and quality along with the z char for security (1x1+2*cov)
                reads = DNA_seqs.get_nuc(i, COV_TEST, npos)
                quality = DNA_seqs.get_quality(i, COV_TEST, npos)
                to_add = np.hstack([reads,quality])
                if quality.max() != 126:
                    # print(f"Debug: {quality.max()} and sequence {i}, nucleotide {npos}")
                    test.append(to_add)
                    seq.append(i)
                    
#3.64 mins to run for COV_TEST=2, CHAR_TEST='z'
#2.87 mins to run for COV_TEST=3, ""


KeyboardInterrupt: 

In [ ]:
len(test)
cov = 2


q_test = 73
 
areas = []
for i in range(len(test)):
    if (test[i][slice(cov,2*cov)].max() < q_test): areas.append(seq[i])

subpar_reads = [test[i] for i in range(len(test)) if all(x < q_test for x in test[i][slice(cov,2*cov)])]
# subpar_reads = [test[i] for i in range(len(test)) if test[i][3] < q_test and test[i][2] < q_test]
len(subpar_reads), subpar_reads, areas

In [99]:
COV_TEST=1
CHAR_TEST='x' # only x for this cell

test = [] 
seq = []

for i in totalDS: #whole 
    cov = DNA_seqs.get_coverage_count(i)
    if (cov == COV_TEST):  # Only care about a specific coverage
        interp = DNA_seqs.get_interp_mutations(i, COV_TEST) # Now we're looking at the interp changes... eventually searching for 'z'
        for npos in range(len(interp)):
            if interp[npos] == ord(CHAR_TEST): #Oh dang there's the char at this npos... let's add an array showing the nuc reads and quality along with the z char for security (1x1+2*cov)
                reads = DNA_seqs.get_nuc(i, COV_TEST, npos)
                quality = DNA_seqs.get_quality(i, COV_TEST, npos)
                to_add = np.hstack([reads,quality])
                if quality.max() != 126:
                    # print(f"Debug: {quality.max()} and sequence {i}, nucleotide {npos}")
                    test.append(to_add)
                    seq.append(i)
len(test)



405665

In [101]:
len(test)
cov = 1

q_test = 73
 
areas = []
for i in range(len(test)):
    if (test[i][slice(cov,2*cov)].max() < q_test): areas.append(seq[i])

subpar_reads = [test[i] for i in range(len(test)) if all(x < q_test for x in test[i][slice(cov,2*cov)])]
# subpar_reads = [test[i] for i in range(len(test)) if test[i][3] < q_test and test[i][2] < q_test]
len(subpar_reads), subpar_reads, areas[0:5]

(159856,
 [array([67, 38], dtype=uint16),
  array([84, 36], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([65, 68], dtype=uint16),
  array([65, 57], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([67, 47], dtype=uint16),
  array([65, 51], dtype=uint16),
  array([71, 56], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([71, 71], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([71, 42], dtype=uint16),
  array([65, 45], dtype=uint16),
  array([67, 51], dtype=uint16),
  array([67, 61], dtype=uint16),
  array([67, 49], dtype=uint16),
  array([84, 40], dtype=uint16),
  array([84, 51], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
  array([45, 33], dtype=uint16),
 

In [68]:
# I want to test the get_nuc and get_quality funcs
nuc_test=1
sb_test=47

true = seqblock_parser(DNA_seqs.get_seqblock(sb_test))

cov = DNA_seqs.get_coverage_count(sb_test)
reads= DNA_seqs.get_nuc(sb_test, cov, nuc_test)
quality = DNA_seqs.get_quality(sb_test, cov, nuc_test)
reads, quality, cov


(array([84, 84], dtype=uint16), array([126, 126], dtype=uint16), 2)

In [90]:
ord('A'),ord('C'),ord('G'),ord('T'), ord('-'), ord('!')

(65, 67, 71, 84, 45, 33)

In [33]:
def pscore(Qscore): 
    return pow(10, -(Qscore-33)/10)
pscore(42)

0.12589254117941673

In [91]:
ord('N')

78